In [1]:
import geopandas as gpd
import pandas as pd
import os
from shapely.geometry import Point, LineString

# busroute.shp 的 欄位名稱(因每次資料來源提供的會有所不同)
route_routename_col = 'RouteNameZ'
route_direction_col = 'Direction'

# Seq 的欄位名稱 (因每次資料來源提供的會有所不同)
seq_routename_col = 'RouteName'
seq_direction_col = 'Direction'
seq_seq_col = 'Seq'
seq_lat_col = 'Lat'
seq_lng_col = 'Lon'

busroute = gpd.read_file(os.path.join(os.getcwd(),'..', 'input','Shp','BusRoute.shp'))
seq = seq = pd.read_csv(os.path.join(os.getcwd(),'..', 'input','seq.csv'))

In [2]:
# 只做同時有站序 & 路線檔案的
routelist = list(set(list(busroute[route_routename_col])) & set(list(seq[seq_routename_col])))
print("可計算的路線共有:", len(routelist),'條')
only_in_route = list(set(busroute[route_routename_col]) - set(seq[seq_routename_col]))
print("只有路線檔案的路線:", only_in_route)
only_in_seq = list(set(seq[seq_routename_col]) - set(busroute[route_routename_col]))
print("只有站序檔案的路線:", only_in_seq)

可計算的路線共有: 379 條
只有路線檔案的路線: []
只有站序檔案的路線: []


In [3]:
def split_line(line):
    """將長的 LineString 拆解為多個小段"""
    coords = list(line.coords)
    segments = []
    for i in range(len(coords) - 1):
        segment = LineString([coords[i], coords[i + 1]])
        segments.append(segment)
    return segments

# 2. 找出最近的路線段的函數
def find_nearest_segment(point, line_segments):
    """找出站點與所有路線段中距離最近的那條路線段"""
    min_distance = float('inf')
    nearest_segment = None
    for segment in line_segments:
        distance = point.distance(segment)
        if distance < min_distance:
            min_distance = distance
            nearest_segment = segment
    return nearest_segment

# 3. 主函數：將路線與站點結合
def get_busroute_segment(busroute_select, seq_select, seq_seq_col):
    """
    這個函數會拆解公車路線，並將每個站點黏貼到最近的路線段上，
    最後依據站點的自定義順序欄位排序並建立每段路線的 GeoDataFrame。

    :param busroute_select: 包含公車路線的 GeoDataFrame
    :param seq_select: 包含公車站點的 GeoDataFrame
    :param seq_seq_col: 指定用於排序的站點順序欄位名稱
    :return: 每段路線的 GeoDataFrame (gdf_segments)
    """
    
    # 1. 拆解 busroute_select 中的路線資料
    busroute_select['geometry'] = busroute_select['geometry'].apply(
        lambda x: split_line(x) if isinstance(x, LineString) else x
    )
    
    # 2. 將站點與最近的路線段進行匹配
    seq_select['nearest_segment'] = seq_select['geometry'].apply(
        lambda point: find_nearest_segment(point, busroute_select['geometry'][0])
    )
    
    # 3. 依據 seq_seq_col 排序站點
    seq_select = seq_select.sort_values(by=seq_seq_col)
    
    # 4. 創建每段路線的 GeoDataFrame
    segments = []
    for i in range(len(seq_select) - 1):
        start_stop = seq_select.iloc[i]
        end_stop = seq_select.iloc[i + 1]
        
        # 連接兩個站點
        segment = {
            'StartSeq': start_stop[seq_seq_col],
            'EndSeq': end_stop[seq_seq_col],
            'geometry': LineString([start_stop['geometry'], end_stop['geometry']])
        }
        segments.append(segment)
    
    # 創建 GeoDataFrame 來儲存每段路線
    gdf_segments = gpd.GeoDataFrame(segments, geometry='geometry', crs="EPSG:4326")
    gdf_segments = gdf_segments[gdf_segments['StartSeq'] < gdf_segments['EndSeq']].reset_index(drop = True)
    gdf_segments = gdf_segments.sort_values(['StartSeq', 'EndSeq'])
    gdf_segments['OD'] = gdf_segments['StartSeq'].astype(str) + "-" + gdf_segments['EndSeq'].astype(str)
    return gdf_segments


以下嘗試

In [4]:
shp_outputfolder = os.path.join(os.getcwd(),'..', 'output','shp')
os.makedirs(shp_outputfolder, exist_ok=True)

In [5]:
route = routelist[0]
print("現在示範的路線編號為：", route)

directions = [0,1]
direction = directions[0]
print("現在示範的方向為：", direction)

現在示範的路線編號為： L602A
現在示範的方向為： 0


In [6]:
all_segments = []

for route in routelist[:5]:  # 限制為前 3 條路線進行測試
    for direction in [0, 1]:  # 方向 0 和 1
        try:
            # 選取特定路線和方向的資料
            busroute_select = busroute[
                (busroute[route_routename_col] == route) & 
                (busroute[route_direction_col] == direction)
            ][[route_routename_col, route_direction_col, 'geometry']].reset_index(drop=True)
            
            seq_select = seq[
                (seq[seq_routename_col] == route) & 
                (seq[seq_direction_col] == direction)
            ].sort_values(seq_seq_col).reset_index(drop=True)
            
            # 將站點轉換為點的 geometry 格式
            seq_select['geometry'] = seq_select.apply(
                lambda row: Point(row[seq_lng_col], row[seq_lat_col]), axis=1
            )
            
            # 使用函數計算分段路線
            gdf_segments = get_busroute_segment(busroute_select, seq_select, seq_seq_col=seq_seq_col)
            
            # 加入路線名稱與方向欄位
            gdf_segments['RouteName'] = route
            gdf_segments['Direction'] = direction
            
            # 將分段結果加入 all_segments 列表
            all_segments.append(gdf_segments)
        
        except:
            pass

# 將所有分段結果合併成一個 GeoDataFrame
all_segments_gdf = gpd.GeoDataFrame(pd.concat(all_segments, ignore_index=True), crs="EPSG:4326")
all_segments_gdf.to_file(os.path.join(shp_outputfolder, 'segment_route.shp'))


C:\Users\kjchang\AppData\Local\Temp\ipykernel_24976\2327857774.py:35: UserWarning: Geometry column does not contain geometry.
  busroute_select['geometry'] = busroute_select['geometry'].apply(
C:\Users\kjchang\AppData\Local\Temp\ipykernel_24976\2327857774.py:35: UserWarning: Geometry column does not contain geometry.
  busroute_select['geometry'] = busroute_select['geometry'].apply(
C:\Users\kjchang\AppData\Local\Temp\ipykernel_24976\2327857774.py:35: UserWarning: Geometry column does not contain geometry.
  busroute_select['geometry'] = busroute_select['geometry'].apply(
C:\Users\kjchang\AppData\Local\Temp\ipykernel_24976\2327857774.py:35: UserWarning: Geometry column does not contain geometry.
  busroute_select['geometry'] = busroute_select['geometry'].apply(
C:\Users\kjchang\AppData\Local\Temp\ipykernel_24976\2327857774.py:35: UserWarning: Geometry column does not contain geometry.
  busroute_select['geometry'] = busroute_select['geometry'].apply(
C:\Users\kjchang\AppData\Local\Temp

In [7]:
# busroute_select = busroute[ (busroute[route_routename_col] == route) & (busroute[route_direction_col] == direction)][[route_routename_col,route_direction_col,'geometry' ]].reset_index(drop = True)
# seq_select = seq[ (seq[seq_routename_col] == route) & (seq[seq_direction_col] == direction) ].sort_values(seq_seq_col).reset_index(drop = True)
# # 假設 seq_select 包含 Lat 和 Lon 欄位，創建一個 geometry 欄位
# seq_select['geometry'] = seq_select.apply(lambda row: Point(row[seq_lng_col], row[seq_lat_col]), axis=1)

In [8]:
# gdf_segments = get_busroute_segment(busroute_select, seq_select, seq_seq_col='Seq')
# gdf_segments['RouteName'] = route


In [9]:
# gdf_segments

拆分

In [10]:
# # 1. 拆解公車路線的函數

# import geopandas as gpd
# import pandas as pd
# import os
# from shapely.geometry import Point, LineString

# def split_line(line):
#     """將長的 LineString 拆解為多個小段"""
#     coords = list(line.coords)
#     segments = []
#     for i in range(len(coords) - 1):
#         segment = LineString([coords[i], coords[i + 1]])
#         segments.append(segment)
#     return segments

# # 2. 找出最近的路線段的函數
# def find_nearest_segment(point, line_segments):
#     """找出站點與所有路線段中距離最近的那條路線段"""
#     min_distance = float('inf')
#     nearest_segment = None
#     for segment in line_segments:
#         distance = point.distance(segment)
#         if distance < min_distance:
#             min_distance = distance
#             nearest_segment = segment
#     return nearest_segment

# # 3. 主函數：將路線與站點結合
# def get_busroute_segment(busroute_select, seq_select, seq_seq_col):
#     """
#     這個函數會拆解公車路線，並將每個站點黏貼到最近的路線段上，
#     最後依據站點的自定義順序欄位排序並建立每段路線的 GeoDataFrame。

#     :param busroute_select: 包含公車路線的 GeoDataFrame
#     :param seq_select: 包含公車站點的 GeoDataFrame
#     :param seq_seq_col: 指定用於排序的站點順序欄位名稱
#     :return: 每段路線的 GeoDataFrame (gdf_segments)
#     """
    
#     # 1. 拆解 busroute_select 中的路線資料
#     busroute_select['geometry'] = busroute_select['geometry'].apply(
#         lambda x: split_line(x) if isinstance(x, LineString) else x
#     )
    
#     # 2. 將站點與最近的路線段進行匹配
#     seq_select['nearest_segment'] = seq_select['geometry'].apply(
#         lambda point: find_nearest_segment(point, busroute_select['geometry'][0])
#     )
    
#     # 3. 依據 seq_seq_col 排序站點
#     seq_select = seq_select.sort_values(by=seq_seq_col)
    
#     # 4. 創建每段路線的 GeoDataFrame
#     segments = []
#     for i in range(len(seq_select) - 1):
#         start_stop = seq_select.iloc[i]
#         end_stop = seq_select.iloc[i + 1]
        
#         # 連接兩個站點
#         segment = {
#             'StartSeq': start_stop[seq_seq_col],
#             'EndSeq': end_stop[seq_seq_col],
#             'geometry': LineString([start_stop['geometry'], end_stop['geometry']])
#         }
#         segments.append(segment)
    
#     # 創建 GeoDataFrame 來儲存每段路線
#     gdf_segments = gpd.GeoDataFrame(segments, geometry='geometry', crs="EPSG:4326")
#     gdf_segments = gdf_segments[gdf_segments['StartSeq'] < gdf_segments['EndSeq']].reset_index(drop = True)
#     gdf_segments = gdf_segments.sort_values(['StartSeq', 'EndSeq'])
#     gdf_segments['OD'] = gdf_segments['StartSeq'].astype(str) + "-" + gdf_segments['EndSeq'].astype(str)
#     return gdf_segments


In [11]:
# gdf_segments = get_busroute_segment(busroute_select, seq_select, seq_seq_col='Seq')


In [12]:
# gdf_segments

In [13]:
# import geopandas as gpd
# from shapely.geometry import Point, LineString
# import pandas as pd

# # 1. 拆解公車路線的函數
# def split_line(line):
#     """將長的 LineString 拆解為多個小段"""
#     coords = list(line.coords)
#     segments = []
#     for i in range(len(coords) - 1):
#         segment = LineString([coords[i], coords[i + 1]])
#         segments.append(segment)
#     return segments

# # 2. 找出最近的路線段的函數
# def find_nearest_segment(point, line_segments):
#     """找出站點與所有路線段中距離最近的那條路線段"""
#     min_distance = float('inf')
#     nearest_segment = None
#     for segment in line_segments:
#         distance = point.distance(segment)
#         if distance < min_distance:
#             min_distance = distance
#             nearest_segment = segment
#     return nearest_segment

# # 3. 主函數：將路線與站點結合
# def get_busroute_segment(busroute_select, seq_select):
#     """
#     這個函數會拆解公車路線，並將每個站點黏貼到最近的路線段上，
#     最後依據站點的 Seq 排序並建立每段路線的 GeoDataFrame。

#     :param busroute_select: 包含公車路線的 GeoDataFrame
#     :param seq_select: 包含公車站點的 GeoDataFrame
#     :return: 每段路線的 GeoDataFrame (gdf_segments)
#     """
    
#     # 1. 拆解 busroute_select 中的路線資料
#     busroute_select['geometry'] = busroute_select['geometry'].apply(
#         lambda x: split_line(x) if isinstance(x, LineString) else x
#     )
    
#     # 2. 將站點與最近的路線段進行匹配
#     seq_select['nearest_segment'] = seq_select['geometry'].apply(
#         lambda point: find_nearest_segment(point, busroute_select['geometry'][0])
#     )
    
#     # 3. 依據 Seq 排序站點
#     seq_select = seq_select.sort_values(by='Seq')
    
#     # 4. 創建每段路線的 GeoDataFrame
#     segments = []
#     for i in range(len(seq_select) - 1):
#         start_stop = seq_select.iloc[i]
#         end_stop = seq_select.iloc[i + 1]
        
#         # 連接兩個站點
#         segment = {
#             'StartSeq': start_stop['Seq'],
#             'EndSeq': end_stop['Seq'],
#             'geometry': LineString([start_stop['geometry'], end_stop['geometry']])
#         }
#         segments.append(segment)
    
#     # 創建 GeoDataFrame 來儲存每段路線
#     gdf_segments = gpd.GeoDataFrame(segments, geometry='geometry', crs="EPSG:4326")
    
#     return gdf_segments


In [14]:
# import geopandas as gpd
# from shapely.geometry import Point, LineString
# import pandas as pd

# # 假設 busroute_select 是你已經匯入的公車路線 GeoDataFrame
# # 假設 seq_select 是你已經匯入的站點 GeoDataFrame

# # 1. 拆解公車路線
# def split_line(line):
#     """將長的 LineString 拆解為多個小段"""
#     coords = list(line.coords)
#     segments = []
#     for i in range(len(coords) - 1):
#         segment = LineString([coords[i], coords[i + 1]])
#         segments.append(segment)
#     return segments

# # 拆解 busroute_select 中的路線資料
# busroute_select['geometry'] = busroute_select['geometry'].apply(lambda x: split_line(x) if isinstance(x, LineString) else x)

# # 2. 站點黏貼到最近的路線段
# # 計算每個站點到每條路線段的距離並找出最近的路線段
# def find_nearest_segment(point, line_segments):
#     """找出站點與所有路線段中距離最近的那條路線段"""
#     min_distance = float('inf')
#     nearest_segment = None
#     for segment in line_segments:
#         distance = point.distance(segment)
#         if distance < min_distance:
#             min_distance = distance
#             nearest_segment = segment
#     return nearest_segment

# # 將站點與最近的路線段進行匹配
# seq_select['nearest_segment'] = seq_select['geometry'].apply(lambda point: find_nearest_segment(point, busroute_select['geometry'][0]))

# # 3. 依據 Seq 排序站點，並建立每段路線
# seq_select = seq_select.sort_values(by='Seq')

# # 創建每段路線的 GeoDataFrame
# segments = []
# for i in range(len(seq_select) - 1):
#     start_stop = seq_select.iloc[i]
#     end_stop = seq_select.iloc[i + 1]
    
#     # 連接兩個站點
#     segment = {
#         'StartSeq': start_stop['Seq'],
#         'EndSeq': end_stop['Seq'],
#         'geometry': LineString([start_stop['geometry'], end_stop['geometry']])
#     }
#     segments.append(segment)

# # 創建 GeoDataFrame 來儲存每段路線
# gdf_segments = gpd.GeoDataFrame(segments, geometry='geometry', crs="EPSG:4326")



In [15]:
# gdf_segments['OD'] = gdf_segments['StartSeq'].astype(str) + "-" + gdf_segments['EndSeq'].astype(str)

In [16]:

# gdf_segments.to_file(os.path.join(shp_outputfolder, 'select_segment_route.shp'))
# seq_select.to_csv(os.path.join(shp_outputfolder, 'select_seq.csv'))

In [17]:
# import folium
# from folium import plugins

# # 設定地圖的初始位置
# map_center = [seq_select['Lat'].mean(), seq_select['Lon'].mean()]
# m = folium.Map(location=map_center, zoom_start=15)

# # 1. 添加站點的標記
# for idx, row in seq_select.iterrows():
#     folium.Marker(
#         location=[row['Lat'], row['Lon']],
#         popup=row['StopName'],
#         icon=folium.Icon(color='red', icon='info-sign')
#     ).add_to(m)

# # 2. 添加路線段
# for i in range(len(seq_select) - 1):
#     start = seq_select.iloc[i]
#     end = seq_select.iloc[i + 1]
    
#     # 連接兩個站點
#     folium.PolyLine(
#         locations=[[start['Lat'], start['Lon']], [end['Lat'], end['Lon']]],
#         color='blue',
#         weight=2.5,
#         opacity=1
#     ).add_to(m)

# # 顯示地圖
# m.save("bus_route_map.html")
